***BRAIN***

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#!pip install --user pipx
#!pip install pipenv


In [2]:
#Building the brain class
class Brain():
    def __init__(self, numInputs, numOutputs, lr):
        self.numInputs=numInputs
        self.numOutputs=numOutputs
        self.learningRate=lr
        self.model=Sequential()
        
        self.model.add(Dense(32,activation='relu',input_shape=(self.numInputs,)))
        self.model.add(Dense(16,activation='relu'))
        self.model.add(Dense(self.numOutputs))
        self.model.compile(optimizer=Adam(learning_rate=self.learningRate),loss='mean_squared_error')
        

***Initializing Experience Replay Memory***

In [3]:
#Initializing Experience Replay Memory
import numpy as np
class Dqn():
    def __init__(self,maxMemory,gamma):
        self.maxMemory=maxMemory
        self.gamma=gamma
        self.memory=list()
    #remembering new experience
    def remember(self, transition, gameOver):
        self.memory.append([transition,gameOver])
        if len(self.memory)>self.maxMemory:
            del self.memory[0]
    #Getting batches of inputs and targets
    def getBatch(self, model, batchSize):
        lenMemory=len(self.memory)
        numInputs=self.memory[0][0][0].shape[1]
        numOutputs=model.output_shape[-1]
        #Initializing the inputs and targets
        inputs=np.zeros((min(batchSize,lenMemory),numInputs))
        targets=np.zeros((min(batchSize,lenMemory),numOutputs))
        #Extracting transitions from random experiences
        for i,inx in enumerate(np.random.randint(0,lenMemory,size=min(batchSize,lenMemory))):
            currentState,action,reward,nextState=self.memory[inx][0]
            gameOver=self.memory[inx][1]
            #Updating Inputs and targets
            inputs[i]=currentState
            targets[i]=model.predict(currentState)[0]
            if gameOver:
                targets[i][action]=reward
            else:
                targets[i][action]=reward+self.gamma* np.max(model.predict(nextState)[0])
        return inputs,targets
    

***Traning***

In [4]:
import gym
import matplotlib.pyplot as plt

learningRate=0.001
maxMemory=5000
gamma=0.9
batchSize=12
epsilon=1.
epsilonDecayRate=0.995

In [5]:
env=gym.make('MountainCar-v0')
#env.mode='fast'
brain=Brain(2,3,learningRate)
model=brain.model
DQN=Dqn(maxMemory,gamma)

In [ ]:
epoch =0
currentState=np.zeros((1,2))
nextState=currentState
totReward=0
rewards=list()
while True:
    epoch+=1
    env.reset()
    currentState=np.zeros((1,2))
    nextState=currentState
    gameOver=False
    while not gameOver:
        if np.random.rand()<=epsilon:
            action=np.random.randint(0,3)
        else:
            qvalues=model.predict(currentState)[0]
            action=np.argmax(qvalues)
        nextState[0],reward,gameOver,_=env.step(action)
        env.render()
        totReward+=reward
        DQN.remember([currentState,action,reward,nextState],gameOver)
        inputs,targets=DQN.getBatch(model,batchSize)
        model.train_on_batch(inputs,targets)
        currentState=nextState
    epsilon*=epsilonDecayRate
    print('Epoch:'+str(epoch)+'Epsilon:{:.5f}'.format(epsilon)+'Total Rewards:{:.2f}'.format(totReward))
    rewards.append(totReward)
    '''totReward
    plt.plot(rewards)
    plt.xlabel('Epoch')
    plt.ylabel('Rewards')
    plt.show()'''
env.close()

Epoch:1Epsilon:0.99500Total Rewards:-200.00
Epoch:2Epsilon:0.99003Total Rewards:-400.00
Epoch:3Epsilon:0.98507Total Rewards:-600.00
Epoch:4Epsilon:0.98015Total Rewards:-800.00
Epoch:5Epsilon:0.97525Total Rewards:-1000.00
Epoch:6Epsilon:0.97037Total Rewards:-1200.00
Epoch:7Epsilon:0.96552Total Rewards:-1400.00
Epoch:8Epsilon:0.96069Total Rewards:-1600.00
Epoch:9Epsilon:0.95589Total Rewards:-1800.00
Epoch:10Epsilon:0.95111Total Rewards:-2000.00
Epoch:11Epsilon:0.94635Total Rewards:-2200.00
Epoch:12Epsilon:0.94162Total Rewards:-2400.00
Epoch:13Epsilon:0.93691Total Rewards:-2600.00
Epoch:14Epsilon:0.93223Total Rewards:-2800.00
Epoch:15Epsilon:0.92757Total Rewards:-3000.00
Epoch:16Epsilon:0.92293Total Rewards:-3200.00
Epoch:17Epsilon:0.91832Total Rewards:-3400.00
Epoch:18Epsilon:0.91372Total Rewards:-3600.00
Epoch:19Epsilon:0.90916Total Rewards:-3800.00
Epoch:20Epsilon:0.90461Total Rewards:-4000.00
Epoch:21Epsilon:0.90009Total Rewards:-4200.00
Epoch:22Epsilon:0.89559Total Rewards:-4400.00
E

Epoch:176Epsilon:0.41387Total Rewards:-35200.00
Epoch:177Epsilon:0.41180Total Rewards:-35400.00
Epoch:178Epsilon:0.40974Total Rewards:-35600.00
Epoch:179Epsilon:0.40769Total Rewards:-35800.00
Epoch:180Epsilon:0.40565Total Rewards:-36000.00
Epoch:181Epsilon:0.40362Total Rewards:-36200.00
Epoch:182Epsilon:0.40161Total Rewards:-36400.00
Epoch:183Epsilon:0.39960Total Rewards:-36600.00
Epoch:184Epsilon:0.39760Total Rewards:-36800.00
Epoch:185Epsilon:0.39561Total Rewards:-37000.00
Epoch:186Epsilon:0.39363Total Rewards:-37200.00
Epoch:187Epsilon:0.39167Total Rewards:-37400.00
Epoch:188Epsilon:0.38971Total Rewards:-37600.00
Epoch:189Epsilon:0.38776Total Rewards:-37800.00
Epoch:190Epsilon:0.38582Total Rewards:-38000.00
Epoch:191Epsilon:0.38389Total Rewards:-38200.00
Epoch:192Epsilon:0.38197Total Rewards:-38400.00
Epoch:193Epsilon:0.38006Total Rewards:-38600.00
Epoch:194Epsilon:0.37816Total Rewards:-38800.00
Epoch:195Epsilon:0.37627Total Rewards:-39000.00
Epoch:196Epsilon:0.37439Total Rewards:-3

Epoch:347Epsilon:0.17563Total Rewards:-69337.00
Epoch:348Epsilon:0.17476Total Rewards:-69537.00
Epoch:349Epsilon:0.17388Total Rewards:-69737.00
Epoch:350Epsilon:0.17301Total Rewards:-69937.00
Epoch:351Epsilon:0.17215Total Rewards:-70137.00
Epoch:352Epsilon:0.17129Total Rewards:-70337.00
Epoch:353Epsilon:0.17043Total Rewards:-70537.00
Epoch:354Epsilon:0.16958Total Rewards:-70737.00
Epoch:355Epsilon:0.16873Total Rewards:-70937.00
Epoch:356Epsilon:0.16789Total Rewards:-71137.00
Epoch:357Epsilon:0.16705Total Rewards:-71337.00
Epoch:358Epsilon:0.16621Total Rewards:-71490.00
Epoch:359Epsilon:0.16538Total Rewards:-71690.00
Epoch:360Epsilon:0.16455Total Rewards:-71890.00
Epoch:361Epsilon:0.16373Total Rewards:-72090.00
Epoch:362Epsilon:0.16291Total Rewards:-72290.00
Epoch:363Epsilon:0.16210Total Rewards:-72490.00
Epoch:364Epsilon:0.16129Total Rewards:-72690.00
Epoch:365Epsilon:0.16048Total Rewards:-72862.00
Epoch:366Epsilon:0.15968Total Rewards:-73062.00
Epoch:367Epsilon:0.15888Total Rewards:-7